## Introduction

We investigate the nature and construction of both 1 dimensional and 2 dimensional cellular automata as well as it's applications to pseudo-random number generation. Cellular automata are systems which have many different cells that exist either in an on or off state and evolve to the next state over a time step according to a set of predetermined rules. In Task 1 we investigate the nature of single cellular "Wolfram" automata, which consist of an arbitrary number of cells in a row which evolve based on the current state of the adjacent cells. The evolution rules governing the automata can be reduced to a single 8-bit binary integer, and this allowed us to write a program which generates a sequence of 1 dimensional automata for any number of tiem steps.

Using the egnerated sequences of automata, we limit the number of cells to 64 in order to emulate a binary representation of a 64-bit integer. We analyzed the usefulness of cellular automata as a pseudo-random number generator by measureing the Shannon entropy of the generated sequences multiple times, using different definitions of the possible outcomes of the system for the calculation to determine the best approach. The highest entropy rule could then be used to generate psudo-random number sequences.

Finally we also analyzed a 2 dimensional cellular automata known as Conway's Game of Life. By allowing the automata to run for large amounts of time we analyzed the steady state behavior of Conway's Game of Life as it related to the intial starting density of the board.

## Task 1.2

The following code tests the entropy of all 256 possible single cell automata rules to determine which is suitable for use as a random number generator. We test different amounts of successive cells as our events in question, including single, double and triple cell configurations to test the entropy of the rules.

In [112]:
import numpy as np
import matplotlib.pyplot as plt

# Taken from pseudo-random notes
def symbol_entropy(datalist):
    '''Estimate the Shannon entropy for a sequence of symbols.
    '''
    
    hist = {}
    for item in datalist:
        if item not in hist:
            hist[item] = 1
        else: 
            hist[item] += 1
            
    counts = np.array( [hist[item] for item in hist])
    prob = counts / np.sum(counts)
    prob = prob[ prob != 0 ] #; print(prob)
    entropy = -np.sum( prob * np.log2(prob) )

    return entropy

In [113]:
# These are Elijah's functions

def cellular_step(states, rule_number):
    """Applies the specified rule number to the list of states by converting it into binary,
    and using the the 3 bit integer associated with every neighborhood of cells in the 
    list, indexes the appropriate state change from the binary rule number. Classically
    the leading digits in the binary representation of the rule number correspond to state
    changes in the largest integers associated with the state neighborhoods (for instance 
    the left most digit would be the state change for the neighborhood 111, associated with 
    the integer 7), so as larger indexes relate to items further right in an array we invert
    the binary representation of the rule_number.
    
    Arguments:
    states -- a row of states associated for a cellular automaton
    rule_number -- an 8 bit integer associated with a way of obtaining the next
                    state for each cell in states using the values of its neighbors
                    
    Returns:
    states -- the new generation of the states after applying the transition rule
    """
    
    # Turns the integer rule number into an 8 bit binary number, and reverses its order
    transition_rule = np.unpackbits( np.uint8(rule_number) )[::-1]
    
    # Initializes an array for storing the index associated to the transition of each cell
    indices = np.zeros(len(states), dtype = np.uint8)
    
    # Determines the index in the transition rule for each cell in the list of states
    for i in range(len(states)):
        # Converts the triple of ones and zeros associated to any state neighborhood into 
        # a base ten integer. Uses mod len(states) so that the cells on the edges loop around
        # when looking at their neighbors
        indices[i] = states[(i-1) % len(states)]*4+states[i % len(states)]*2 + states[(i+1)%len(states)]
        
        

    # Applies the transition rule to each state using its calculated index
    states = transition_rule[ indices ]

    return states

def run_automaton(rule_number, ngens, ncells):
    """Runs the given rule number on a collection of cells
    of length ncells a specified number of times, and returns
    a grid containing each generation.
    
    Arguments:
    rule_number -- an 8 bit integer used in deciding the next state of each cell in
                    the automaton based on the states of its neighbors and its current state
    ngens -- the number of generations of the automaton to compute
    ncells -- the number of cells in the automaton
                    
    Returns:
    grid -- a collection of each generation of cells in the automaton
    """
    
    # Initializes row of states with a one in the center
    states = np.zeros(ncells, dtype = np.uint8)
    states[len(states)//2] = 1
    
    # Creates a grid for storing the various generations of the cells
    grid = np.zeros((ngens, ncells), dtype = np.uint8)
    grid[1,:] = states
    
    # For each grid, updates the cell states with some cellular_step function and the given rule number
    for i in range(1,ngens):
        states = cellular_step(states, rule_number)
        grid[i,:] = states
    
    return grid

In [114]:
def multi_step_rule_entropy(rule, steps, ncells=64, nevents=400):
    """This function takes in a given 1-D cellular automaton rule and calculates the total 
    entropy for a specific number of events where each event correpsonds to a specific number 
    of successive steps in the state of the automaton.
    
    Arguments:
    rule -- an 8-bit unsigned integer that specified the automata rule in question
    steps -- the number of succesive steps in the automation being considered a single outcome
    ncells -- the number of cells of the automation
    nevents -- the number of outcomes which should be tested for the entropy calculation
    
    Returns:
    entropy/steps -- The function returns the sum of the total entropy of every cell over 
                    the course of the automation per step
    """
    
#     # Create the intial state
#     state = np.zeros(ncells, dtype=np.uint8)
#     state[ncells//2] = 1  # initialize one cell near the middle
    
#     nstates = nevents*steps
#     buffer = 50*steps
#     states = np.zeros([nstates, ncells], dtype=np.uint8)
#     for n in range(nstates + buffer):
#         state = cellular_step(state, rule)
#         if (n >= buffer):
#             states[n - buffer] = state
    
    # Get the states of the automaton
    buffer = 50 # Create a buffer so that the auomata reaches some form of steady state
    ngens = nevents*steps + buffer
    states = run_automaton(rule, ngens, ncells)
    states = states[buffer:,:] #Remove the buffer generations
    
    # Calculate the sum of the entropies of every cell
    entropy = 0
    for i in range(ncells):
        events = states[:,i].reshape(nevents, steps) # Reshape the grid into a single row of all values the cell had
        events_tuple = tuple(tuple(event) for event in events) # Cast to tuple for hashing purposes
        entropy += symbol_entropy(events_tuple) # Calculate the entropy for the cell
    
    return entropy/steps

In [124]:
def multi_step_entropies(steps, ncells=64, nevents=400):
    """This function takes in a set of specified parameters for the single 
    cellular automaton and produces an array of values for the entropy of
    the automaton for every possible one dimensional rule as well as a sorted 
    list of the rules from the highest to the lowest entropy.
    
    Arguments:
    steps -- the number of succesive steps in the automation being considered a single outcome
    ncells -- the number of cells of the automation
    nevents -- the number of outcomes which should be tested for the entropy calculation
    
    Returns:
    sorted_rules -- an array of 256 8-bit unsigned integers corresponding to the single cell automata rules 
                    sorted from highest to lowest entropy
    entropies -- an array of 256 floating point numbers where every entry is the entropy corresponding to 
                the single cell automata rule specified by the index
    """
    
    entropies = np.zeros(256)

    # Calculate and store the entropies for every rule
    for rule in np.array(range(256), dtype=np.uint8):
        entropies[rule] = multi_step_rule_entropy(rule, steps, ncells, nevents)

    # Sort the rules by order of decreasing entropy
    sorted_rules = np.argsort(entropies)[::-1]
    
    return sorted_rules, entropies

In [125]:
# We calculate the entropies for three different numbers of successive steps for comparison
one_step_rules, one_step_entropies = multi_step_entropies(1)
two_step_rules, two_step_entropies = multi_step_entropies(2)
three_step_rules, three_step_entropies = multi_step_entropies(3)

In [117]:
print("The five highest entropy rules for single step entropy calculations are (in order):")
for rule in one_step_rules[:5]:
    print("Rule:", rule, "\t Entropy:", one_step_entropies[rule])

The five highest entropy rules for single step entropy calculations are (in order):
Rule: 127 	 Entropy: 64.0
Rule: 58 	 Entropy: 64.0
Rule: 21 	 Entropy: 64.0
Rule: 23 	 Entropy: 64.0
Rule: 55 	 Entropy: 64.0


In [118]:
print("The five highest entropy rules for two step entropy calculations are (in order):")
for rule in two_step_rules[:5]:
    print("Rule:", rule, "\t Entropy:", two_step_entropies[rule])

The five highest entropy rules for two step entropy calculations are (in order):
Rule: 75 	 Entropy: 63.78261580261772
Rule: 89 	 Entropy: 63.78261580261771
Rule: 101 	 Entropy: 63.757528397172436
Rule: 45 	 Entropy: 63.75752839717243
Rule: 30 	 Entropy: 63.73032042995543


In [126]:
print("The five highest entropy rules for three step entropy calculations are (in order):")
for rule in three_step_rules[:5]:
    print("Rule:", rule, "\t Entropy:", three_step_entropies[rule])

The five highest entropy rules for three step entropy calculations are (in order):
Rule: 89 	 Entropy: 63.73352668657989
Rule: 75 	 Entropy: 63.733526686579886
Rule: 86 	 Entropy: 63.70803033881439
Rule: 30 	 Entropy: 63.70803033881438
Rule: 101 	 Entropy: 63.692013038375684


We can see that as we increase the number of steps considered to be a single outcome for the purposes of calculating the entropy of the system, we also see an increase in the difference of calculated entropies between the different rules. This helps us to distinguish the truly random looking rules from ones with successive patterns. Therefore we use the three step entropy calculations for the next task.

## Task 1.3

This code defines a function which utilizes cellular automata to produce a sequance of random unsigned 64-bit integers.

In [120]:
def packbits64(array):
    """This function operates similar to np.packbits but works on 64-bit unsigned integers
    instead of 8-bit integers. It takes in an array of 64 binary values and outputs the appropriate
    64 bit number.
    
    Arguments:
    array -- An array of 64 binray values to be converted to a 64 bit unsigned integer
    
    Returns:
    integer -- A 64 bit unsigned integer
    """
    
    # Assert the right form for the array
    assert ((len(array) == 64) & (array.dtype == np.uint8) & (max(array) <= 1))
    
    # Calculate the integer
    integer = np.uint64(0)
    for index in range(64):
        integer += array[index]*np.uint64(2**(63 - index))
    
    return integer

def unpackbits64(integer):
    """This function operates similar to np.unpackbits but works on 64-bit unsigned integers
    instead of 8-bit integers. It takes in a 64 bit unsigned integer and outputs an array corresponding
    to it's binary representation.
    
    Arguments:
    integer -- A 64 bit unsigned integer
    
    Returns:
    array -- 64 binary values which constitute the binary representation of the integer
    """
    
    # Assert the right form of the integer
    assert (type(integer) == np.uint64)
    
    # Calculate the binary representation
    array = np.zeros(64, dtype= np.uint8)
    for index in range(64):
        if integer >= np.uint64(2**(63 - index)):
            array[index] = 1
            integer %= np.uint64(2**(63 - index)) # Divide out the value of the digit stored form the integer
    
    return array

In [121]:
def automata_rand(rule, n=1, seed=None):
    """This funtion produces either a single pseudo-random 64 bit unsigned integer 
    or a sequential array of pseudo-random integers as determined from the sequence
    of states produced by a specified single cellular automata. In the abscence of 
    a given seed the automata runs from a single on cell at position 32 for 50 cycles
    before producing pseudo-random values.
    
    Arguments:
    rule -- The single cell automata rule to be used given as an 8-bit unsigned integer
    n -- The number of values to be produced
    seed -- A 64 bit unsigned integer representing the state of the automaton before
            the new values are generated
    
    Returns:
    seed -- A single pseudo-random number produced when n=1 or the new seed for the 
            next generated psuedo-random number
    gen_nums -- An array of length n>1 containing pseudo-random 64 bit unsigned integers
    """
    
    # Initialize a seed from 50 iterations of the automaton with one cell at the center if a seed is not given
    if seed is None:
        seed = automata_rand(rule, 50, np.uint64(2147483648))[49]
        
    # Make sure the seed is a 64 bit unsigned integer
    if type(seed) != np.uint64:
        seed = np.uint64(seed)
        
    # Pass the seed through one cellular automata cycle
    state = unpackbits64(seed)
    state = cellular_step(state, rule)
    seed = packbits64(state)
        
    if n == 1: # Return a single value
        return seed
    else: # Get the next value and append to the array
        gen_nums = np.empty([1], dtype=np.uint64)
        gen_nums[0] = seed
        gen_nums = np.append(gen_nums, automata_rand(rule, n-1, seed))
        return gen_nums
        
    # Function should never reach this point
    assert (False)
    return

In [122]:
random_numbers = automata_rand(three_step_rules[0], 12)

print("Using single cell automata we can produce a sequence of 12 pseudo-random numbers like so:")
print(random_numbers)

Using single cell automata we can produce a sequence of 12 pseudo-random numbers like so:
[ 4154833229990782878 16970595671829032114 11726952154966920500
   874331476871281777  9064866832490989014 14221716280219220294
 15570501625269938718 15844141963174174962 15721986324399091604
 15620934454096914593  6616685541272019727  1893613931168755577]
